# Disclaimer:
DeepSphere.AI developed these
materials based on its team’s expertise
and technical infrastructure, and we
are sharing these materials strictly for
learning and research. These learning
resources may not work on other learning
infrastructures and DeepSphere.AI
advises the learners to use these materials
at their own risk. As needed, we will be
changing these materials without any
notification and we have full ownership
and accountability to make any change
to these materials.

# Import Libraries and Install Dependencies

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from tensorflow.keras.optimizers import SGD
import random

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ravin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ravin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Class ChatBot
File Descriptions :
- DSAI_intents.json - input data as json file
- DSAI_words.pkl - lemmatized words stored as pickle format
- DSAI_classes.pkl - classes are stored as pickle format
- DSAI_chatbot_model.h5 - saved trained model

In [2]:
class ChatBot:

  def __init__(self, intents): #constructor
  #intents - data from json file
    self.vAR_words=[]
    self.vAR_classes = []
    self.vAR_documents = []
    self.vAR_ignore_words = ['?', '!']
    self.vAR_intents = intents
    self.vAR_lemmatizer = WordNetLemmatizer()
    
    for intent in self.vAR_intents['intents']:
      for pattern in intent['patterns']:

        vAR_w = nltk.word_tokenize(pattern) # take each word and tokenize it
        self.vAR_words.extend(vAR_w)
        self.vAR_documents.append((vAR_w, intent['tag']))  # adding documents

        if intent['tag'] not in self.vAR_classes: # adding classes to our class list
            self.vAR_classes.append(intent['tag'])

    self.vAR_words = [self.vAR_lemmatizer.lemmatize(w.lower()) for w in self.vAR_words if w not in self.vAR_ignore_words]
    self.vAR_words = sorted(list(set(self.vAR_words)))
    self.vAR_classes = sorted(list(set(self.vAR_classes)))

    print(len(self.vAR_documents), "documents")
    print(len(self.vAR_classes), "classes", self.vAR_classes)
    print(len(self.vAR_words), "unique lemmatized words", self.vAR_words)

  def store_object(self): 
    #store/dump lemmatized words and classes as pickle file
    pickle.dump(self.vAR_words,open('../Utility/DSAI_Words.pkl','wb'))
    pickle.dump(self.vAR_classes,open('../Utility/DSAI_Classes.pkl','wb'))

  def initialize_training(self):
    # initializing training data
    # prepare training data
    self.vAR_training = []
    vAR_output_empty = [0] * len(self.vAR_classes)

    for doc in self.vAR_documents:
        vAR_bag = []  # initializing bag of words
        vAR_pattern_words = doc[0]  # list of tokenized words for the pattern
        # lemmatize each word - create base word, in attempt to represent related words
        vAR_pattern_words = [self.vAR_lemmatizer.lemmatize(word.lower()) for word in vAR_pattern_words] 

        # create our bag of words array with 1, if word match found in current pattern
        for w in self.vAR_words:
            vAR_bag.append(1) if w in vAR_pattern_words else vAR_bag.append(0)

        # output is a '0' for each tag and '1' for current tag (for each pattern)
        vAR_output_row = list(vAR_output_empty)
        vAR_output_row[self.vAR_classes.index(doc[1])] = 1

        self.vAR_training.append([vAR_bag, vAR_output_row])
    
    random.shuffle(self.vAR_training)  # shuffle our features and turn into np.array
    self.vAR_training = np.array(self.vAR_training)

  def create_train_test_lists(self):
    # create train and test lists.
    # X - patterns, Y - intents
    self.vAR_train_x = list(self.vAR_training[:,0])
    self.vAR_train_y = list(self.vAR_training[:,1])
    print("Training data created")

  def create_model(self):
    # Number of layers - 3 layers.
    # First layer 128 neurons
    # Second layer 64 neurons 
    # 3rd output layer contains vAR_train_y[0] number of neurons
    # equal to number of intents to predict output intent with softmax
    self.vAR_model = Sequential()
    self.vAR_model.add(Dense(128, input_shape=(len(self.vAR_train_x[0]),), activation='relu'))
    self.vAR_model.add(Dropout(0.5))
    self.vAR_model.add(Dense(64, activation='relu'))
    self.vAR_model.add(Dropout(0.5))
    self.vAR_model.add(Dense(len(self.vAR_train_y[0]), activation='softmax'))

  def compile_model(self):
    # Compile model.
    # Optimizer - Stochastic gradient descent (SGD)
    # Nesterov accelerated gradient gives good results for this model
    vAR_sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    self.vAR_model.compile(loss='categorical_crossentropy', optimizer = vAR_sgd, metrics=['accuracy'])

  def fit_and_save_model(self):
    #fitting the model
    # save the model in .h5 format
    vAR_hist = self.vAR_model.fit(np.array(self.vAR_train_x), np.array(self.vAR_train_y), epochs=200, batch_size=5, verbose=1)
    self.vAR_model.save('../Utility/DSAI_Chatbot_Model.h5', vAR_hist)

    print("Model created")

  def RUN_MODEL(self):
    # method to run all methods
    self.store_object() # method call to store classes and words in pickle format
    self.initialize_training() # method call to prepare training data
    self.create_train_test_lists() #method call to prepare Train-Test list of prepared training data
    self.create_model() #method call to create Sequential model of 3 layers
    # Compile the model with SGD Optimizer, Nesterov Accelerator using Categorical Crossentropy loss and accuracy Metric
    self.compile_model()
    self.fit_and_save_model() #method call to fit the model and save the trained model as DSAI_chatbot_model.h5

# Import Data
Input data from DSAI_intents.json is imported

In [3]:
vAR_data_file = open('../Utility/DSAI_Intents.json').read()
vAR_intents = json.loads(vAR_data_file)

# Create Object
Creation of object for class ChatBot with the dataset

In [4]:
vAR_chat_bot_object = ChatBot(vAR_intents)

46 documents
20 classes ['Textbook', 'access', 'ask', 'askorder', 'benefits', 'career', 'complaint', 'courses', 'createaccount', 'demo', 'goodbye', 'greeting', 'help', 'levels', 'name', 'pay', 'prerequisites', 'registration', 'thanks', 'time']
86 unique lemmatized words ["'s", '.', 'a', 'about', 'access', 'account', 'an', 'anyone', 'are', 'at', 'before', 'benefit', 'buy', 'bye', 'call', 'can', 'career', 'certain', 'chapter', 'complaint', 'could', 'course', 'create', 'data', 'demo', 'do', 'doing', 'enroll', 'enrollment', 'for', 'give', 'goodbye', 'hand', 'have', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'in', 'is', 'it', 'later', 'level', 'me', 'meet', 'my', 'name', 'need', 'new', 'offered', 'offering', 'online', 'open', 'order', 'pay', 'please', 'prerequisite', 'raise', 'register', 'registered', 'science', 'see', 'service', 'should', 'specific', 'start', 'support', 'take', 'textbook', 'thank', 'thanks', 'that', 'the', 'there', 'time', 'to', 'want', 'what', 'whats', 'w

# RUN MODEL

In [5]:
vAR_chat_bot_object.RUN_MODEL()

Training data created


C:\Users\ravin\AppData\Local\Temp/ipykernel_10540/3318931033.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.vAR_training = np.array(self.vAR_training)


Epoch 1/200


C:\Users\ravin\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


10/10 [==============================] - 1s 3ms/step - loss: 3.0377 - accuracy: 0.0652
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 3.0211 - accuracy: 0.0435
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7979 - accuracy: 0.2391
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7937 - accuracy: 0.1957
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7218 - accuracy: 0.1739
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 2.5860 - accuracy: 0.1957
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 2.4991 - accuracy: 0.2391
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 2.3840 - accuracy: 0.3478
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2154 - accuracy: 0.3043
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1700 - accuracy: 0.3696
Epoch 11/200
10/10 [=

10/10 [==============================] - 0s 2ms/step - loss: 0.2102 - accuracy: 0.9565
Epoch 84/200
10/10 [==============================] - 0s 3ms/step - loss: 0.3218 - accuracy: 0.9348
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1742 - accuracy: 0.9348
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.2216 - accuracy: 0.9130
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.2041 - accuracy: 0.9348
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.2200 - accuracy: 0.9783
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.2583 - accuracy: 0.9348
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.2244 - accuracy: 0.9130
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1174 - accuracy: 0.9565
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1679 - accuracy: 0.9565
Epoch 93/200


10/10 [==============================] - 0s 2ms/step - loss: 0.0478 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 3ms/step - loss: 0.1769 - accuracy: 0.9565
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1040 - accuracy: 0.9565
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0826 - accuracy: 0.9565
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0765 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0498 - accuracy: 0.9783
Epoch 170/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0806 - accuracy: 0.9783
Epoch 171/200
10/10 [==============================] - 0s 3ms/step - loss: 0.1144 - accuracy: 0.9783
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0528 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0491 - accuracy: 0.9783
Epoc

# Copyright Notice:
Local and international copyright laws protect
this material. Repurposing or reproducing
this material without written approval from
DeepSphere.AI violates the law.

(c) DeepSphere.AI